In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender


In [2]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [4]:
result = []

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [7]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 4 s


In [8]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [9]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 145 ms


### 1.3 Weighted random recommender

In [10]:
def get_items_weights(data):
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular['weight'] = np.log(popular['sales_value'] + 0.000000001)
    popular['weight'] = np.where(popular['weight'] < 0 , 0, popular['weight'])
    sum_ = popular['weight'].sum()
    popular['weight'] = popular['weight']/sum_  
    popular.drop('sales_value', axis=1, inplace=True)
    return popular

def weighted_random_recommendation(items_weights, n=5):
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])        
    return recs.tolist()

In [11]:
%%time
items_weights = get_items_weights(data_train)

result['weighted_random_recommendation'] = result['user_id'].\
    apply(lambda x: weighted_random_recommendation(items_weights, n=5))


Wall time: 2.15 s


In [12]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [13]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [14]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\spvag\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [15]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [16]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 848 ms


In [18]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[ itemid_to_id[999999] ],
                                    recalculate_user=True)
                    ])

Wall time: 76 ms


In [19]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)


Wall time: 827 ms


In [20]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])


Wall time: 87 ms


In [21]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)


Wall time: 837 ms


In [22]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

Wall time: 92 ms


In [23]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)


Wall time: 802 ms


In [24]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

Wall time: 62 ms


### Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?

In [25]:
data_train_at_5000 = data_train[data_train['item_id']!=999999]

In [26]:
data_train.shape, data_train_at_5000.shape

((2278490, 12), (1448716, 12))

In [27]:
def prepocessing(data_train):
    user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

    user_item_matrix[user_item_matrix > 0] = 1 
    user_item_matrix = user_item_matrix.astype(float) 

    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values
    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))
    dicts = {}
    dicts['id_to_itemid'] = dict(zip(matrix_itemids, itemids))
    dicts['id_to_userid'] = dict(zip(matrix_userids, userids))
    dicts['itemid_to_id'] = dict(zip(itemids, matrix_itemids))
    dicts['userid_to_id'] = dict(zip(userids, matrix_userids))    
    return csr_matrix(user_item_matrix).tocsr(), dicts


csr_matrix_at_5000, encoding = prepocessing(data_train_at_5000)

в силу того что мы удалили часть информации, возможен случай когда в тестовой выборке буду присутствовать юзеры котороых нет в трайне. Предусмотрим такой слуай и будем рекомендовать им случайные рекомендации  

In [28]:
def recommend(user_id):
    if encoding['userid_to_id'].get(user_id) ==None:
        return result.loc[result['user_id']==user_id]['random_recommendation'] 
    res = [encoding['id_to_itemid'][rec[0]] for rec in model.recommend(userid=encoding['userid_to_id'][user_id], 
                                    user_items=csr_matrix_at_5000,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    recalculate_user=True)
                    ]
    return res

In [29]:
%%time

model = ItemItemRecommender(K=5, num_threads=4)
model.fit((csr_matrix_at_5000).T.tocsr(), 
          show_progress=True)
result['itemitem_at_5000'] = result['user_id'].apply(lambda x: recommend(x))


Wall time: 649 ms


In [30]:
%%time

model = CosineRecommender(K=5, num_threads=4)
model.fit(csr_matrix_at_5000.T.tocsr(), 
          show_progress=True)
result['cosine_at_5000'] = result['user_id'].apply(lambda x: recommend(x))


Wall time: 604 ms


In [31]:
%%time

model = TFIDFRecommender(K=5, num_threads=4)
model.fit(csr_matrix_at_5000.T.tocsr(), 
          show_progress=True)
result['tfidf_at_5000'] = result['user_id'].apply(lambda x: recommend(x))


Wall time: 643 ms


In [32]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) 
model.fit(csr_matrix_at_5000.T.tocsr(), 
          show_progress=True)
result['own_purchases_at_5000'] = result['user_id'].apply(lambda x: recommend(x))


Wall time: 552 ms


### Анаплиз полученных данных

In [33]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
 
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [34]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    
    return recall

In [35]:
result.columns[2:]

Index(['random_recommendation', 'popular_recommendation',
       'weighted_random_recommendation', 'itemitem', 'cosine', 'tfidf',
       'own_purchases', 'itemitem_at_5000', 'cosine_at_5000', 'tfidf_at_5000',
       'own_purchases_at_5000'],
      dtype='object')

In [36]:
precision_ = []
for clo in result.columns[2:]:
    precision_.append(result.apply(lambda row: precision_at_k(row[clo], row['actual']), axis=1).mean())

<ipython-input-33-3ffa33a2cae8>:11: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


In [37]:
recol_ = []
for clo in result.columns[2:]:
    recol_.append(result.apply(lambda row: recall_at_k(row[clo], row['actual']), axis=1).mean())

In [38]:
res = pd.DataFrame([list(result.columns[2:]), precision_, recol_]).T

In [39]:
res.columns= ['method','precision_at_k', 'recall_at_k']

In [40]:
res

,method,precision_at_k,recall_at_k
0,random_recommendation,0.000587659,2.50426e-05
1,popular_recommendation,0.15524,0.0249956
2,weighted_random_recommendation,0.000783546,5.73638e-05
3,itemitem,0.151224,0.0181559
4,cosine,0.155142,0.0173445
5,tfidf,0.160921,0.0183568
6,own_purchases,0.201917,0.0293013
7,itemitem_at_5000,0.154946,0.017269
8,cosine_at_5000,0.154261,0.0172251
9,tfidf_at_5000,0.158276,0.017608


Выводы:
   - По полученным результатам лучшим оказался метод "own_purchases_at_5000", хотя выигрыш здесь не значительный и может оказаться статистической пограшностью. Об этом так же говоят и все другие полученные данные, показывая отстутствие каких либо существенных отличий между вариантами обучения на польной выборке и на топ5000. Так же нет значительных отличий и во времени выполнения. 
   - Поэтому для данной задачи и для данной выборки улучшить бейзлайны, если считать их на топ-5000 товарах нельзя